In [2]:
import torch
from torch.autograd import Variable
import time

# Construct a Combined Loss

### Work with examples

In [17]:
output = torch.FloatTensor([[0.8, 0.2, 0.1, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]])
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])

In [18]:
p0 = target[:,0]
p1 = target[:,1]
p2 = target[:,2]
print(p0);print(p1);print(p2)

tensor([1., 0., 0.])
tensor([0., 1., 0.])
tensor([0., 0., 1.])


In [19]:
output[:,0]

tensor([0.8000, 0.4000, 0.0500])

In [20]:
torch.mul(output[:,0], p0)

tensor([0.8000, 0.0000, 0.0000])

In [21]:
logErr0 = torch.mul(torch.log(output[:,0]), p0)
logErr1 = torch.mul(torch.log(output[:,1]), p1)
logErr2 = torch.mul(torch.log(output[:,2]), p2)

In [22]:
mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)

In [23]:
logErr0 = -1 * torch.sum(logErr0)
logErr1 = -1 * torch.sum(logErr1)
logErr2 = -1 * torch.sum(logErr2)
mseErr1 = torch.sum(mseErr1)    
mseErr2 = torch.sum(mseErr2)    

In [24]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

### Define error as function and apply autograd

In [38]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr0 + logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

In [39]:
from torch.autograd import Variable

output = Variable(torch.FloatTensor([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]]), requires_grad=True)
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
err = comb_error(output, target, 100, 100)
err

tensor(1.1647, grad_fn=<MulBackward0>)

In [40]:
err.backward()

In [41]:
print(output.grad)

tensor([[-0.8331,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.6665,  0.0000,  0.0320,  0.0000],
        [ 0.0000,  0.0000, -0.6665,  0.0000, -0.0652]])


## Tests

In [42]:
def comb_error(output, target, sig1, sig2):
    output = (output + 0.0001)*0.999
    p0 = target[:,0]
    p1 = target[:,1]
    p2 = target[:,2]
    
    logErr0 = torch.mul(torch.log(output[:,0]), p0)
    logErr1 = torch.mul(torch.log(output[:,1]), p1)
    logErr2 = torch.mul(torch.log(output[:,2]), p2)
    mseErr1 = torch.mul(torch.pow(target[:,2+1] - output[:,2+1], 2), p1)
    mseErr2 = torch.mul(torch.pow(target[:,2+2] - output[:,2+2], 2), p2)   
    
    logErr0 = -1 * torch.sum(logErr0)
    logErr1 = -1 * torch.sum(logErr1)
    logErr2 = -1 * torch.sum(logErr2)
    mseErr1 = torch.sum(mseErr1)    
    mseErr2 = torch.sum(mseErr2)      
    
    return (1/output.shape[0]) * (logErr0 + logErr1 + logErr2 + (1/sig1)*mseErr1 + (1/sig2)*mseErr2)    

In [43]:
output = Variable(torch.FloatTensor([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]]), requires_grad=True)
target = torch.FloatTensor([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
err = comb_error(output, target, 100, 100)
err

tensor(1.1647, grad_fn=<MulBackward0>)

In [48]:
import numpy as np
repeats = 1000000
yh = np.array([[0.4, 0.4, 0.2, 5.8, 15.22],[0.4, 0.5, 0.1, 7.8, 20.22],[0.05, 0.45, 0.5, 5.8, 10.22]])
yh = np.repeat(yh, [repeats, repeats, repeats], axis=0)
y = np.array([[1,0,0,0,0],[0,1,0,2.99,0],[0,0,1,0,20]])
y = np.repeat(y, [repeats, repeats, repeats], axis=0)

In [49]:
output = Variable(torch.FloatTensor(yh), requires_grad=True)
target = torch.FloatTensor(y)
start = time.time()
err = comb_error(output, target, 100, 100)
end = time.time()
print(err)
print(end - start)

tensor(1.1647, grad_fn=<MulBackward0>)
0.8297433853149414
